In [1]:
import mmf_setup;mmf_setup.nbinit()

<IPython.core.display.Javascript object>

# Data Formats

Here we discuss a proposal for archiving numerical data to disk.  The use case is generating and archiving simulation data, typically in the form of several relatively large arrays.

## Motivation

We have the following goals:

* **Simple:** The format should not be overly complex.  In particular, the code needed to access and load the data should be relatively straight-forward - especially for reading the data.  Formats like HDF5 do not satisfy this ideal, requiring fairly complex libraries be installed in order to interact with the data.
* **Efficient:** Data storage should be efficient in terms of both space and speed.  Compressed text-files, for example, can be quite efficient for limited-precision or redundant data, but uncompressed binary formats tend to be significantly faster.
* **Human readable/editable:** A human should be able to inspect and, if needed, modify the data.  A use case is preventing old data from becoming stale as the source code changes.  Although the ideal is for the data to be insensitive to code changes, this is not always possible in practice.  It should be relatively straightforward for someone to upgrade an old data-set so it can be used with be versions of the code.  Another use case is that a user should be able to make some sense of the data if they stumble across it without the underlying code-base.  Thus, the data-format should be somewhat self-describing as discussed next.
* **Self-describing:** The data format should be self-describing as much as possible.  This means that any binary data should have clearly defined meta-data that specifies the size and type of the data, as well as an appropriate specification of what the various dimensions mean, and what are the physical units of the numbers.
* **Accessible from Python, MATLAB, C++, etc.:** Main collaborators use Python and MATLAB, so interfaces for these platforms are needed.  Any binary format should also be well described and easily accessible from core languages like C++, FORTRAN etc. though this is not a priority.
* **Parallel Access:** Often we would like to generate data using a set of parallel processes, so the dataset must allow for several processes to access it simultaneously.  Storing different arrays in different files, for example, would permit this under the restriction that each process work on its own file.  Conflicts accessing the overall meta-data will need to be mitigated somehow with locks.

# Version 1

To meet these needs, we start with the following format.  Later the backend may be adjusted to improve performance, or interoperability (see the [Existing Solutions](#Existing-Solutions) section below).

* Arrays should be stored in a simple, well-documented format.  We choose either text-files (for human convenience) or using the NumPy [NPY data format](https://docs.scipy.org/doc/numpy-dev/neps/npy-format.html) (`*.npy` files) which can also be memory-mapped.  This format is well documented, simple, portable, and has very easy access with the ubiquitous [NumPy](http://www.numpy.org/) that lies at the core of most scientific Python work.

  * [MATLAB Reader for NPY files](https://github.com/kwikteam/npy-matlab).
  * [How to read .npy files in Matlab](https://stackoverflow.com/questions/21553999/how-to-read-npy-files-in-matlab)

* Arrays should be able to be stored in separate files so that concurrent processes can write to each file.
* Metadata will be stored in executable python files generated with the [persist] package.  These are human-readable, and require nothing beyond Python and NumPy to load. *(This assumes that no custom objects are stored.  Storing custom objects will require the code defining the custom classes to be importable.)*

This strategy is very similar to the [Exdir] standard (see the section [Exdir](#Exdir) below), but this requires python 3 and is a little complicated, so we do not yet use this.

[persist]: https://bitbucket.org/mforbes/persist
[HDF5]: https://support.hdfgroup.org/HDF5/
[NETCDF]: https://www.unidata.ucar.edu/software/netcdf/
[Exdir]: https://github.com/CINPLA/exdir/

## Details

### File Structure

### Array

Each array should have the following metadata stored along with it:

* `name`: Each array needs a unique name.  The metadata must associate the name with the appropriate file containing the data.  Sometimes, the arrays may be organized in another type such as a dictionary or list.  We support the following nested types:
   * List of arrays: Here the name might be something like `x[3]` (third element in a list).
   * Dictionary of arrays: Here the name might be something like `x['a']` or `x[(1,2)]`.
* `shape`: Shape of the array, i.e. `(100,100,3)`. This is already stored in the NPY format.
* `dtype`: Datatype for elements in the array.  This is already stored in the NPY format.
* `unit`: Either a unit describing all data in the array, or a tuple of powers which will be used to form the unit as a product of powers of the `units` of each of the dimensions.
* `dimensions`: Each axis of the array should be described by an appropriate dimension.

### Dimension

A dimension is a description of one of the dimensions of an array.  At the minimum, it should have some sort of name, and a set of "values" for each entry in that array.  We provide two types of dimensions here: a numerical coordinate, and a parameter dimension.  Numerical dimensions must have an associated unit, as well as numerical coordinate values for each point.  Parameter dimensions are instead a list of parameters names as values and a list of units for each parameter.

* `Coordinate` :
  * `name` : Name of the dimension.
  * `coords` : List or array of values associated with the dimension.
  * `unit` : Unit of the values.
* `Parameters` : 
  * `name` : Name of the dimension.
  * `coords` : List of strings naming each parameter.
  * `units` : List of Units of each parameter.
  
Example: Consider a table of $N_m$ masses $M(P_c)$ and radii $R(P_c)$ as tabulated as function of a central pressure $P_c$ for a neutron star (see the LIGO use case below).  These might be represented by a $2 \times N_m$ dimensional `Array` with the following dimensions:

```python
Array(name="MR",
      shape=(2, N_m),
      dtype=float,
      dimensions=[Parameters(name='observable',
                             coords=['M', 'R'],
                             units=['M0', 'km']),
                  Coordinate(name='P_c',
                             coords=np.linspace(P0, P1, N_m),
                             unit='MeV/fm**3')])
```

### Units

Often we would like to be able to automatically visualize data.  To this end, one of the meta-data items should be a mapping that converts names and other strings into symbols.  This mapping should have options for various backends such as Holoview and Matplotlib.  Similarly, each name associated with numerical value should also have a unit.  This should also be stored in the metadata.

* `Unit`: str | (str, Label)

   A `Unit` consists of a string that can be evaluated in an appropriate context to obtain the numerical value of the data, and a `Label` for displaying the unit in plots etc.
   
   Example:
   
   ```python
   ('MeV',   {'ascii': 'MeV', 'LaTeX': '\mathrm{MeV}'})
   ```


# Use Cases

The first use case here is through a project to constrain nuclear parameters with gravitational wave data from LIGO.  A set of $N_p \approx 20$ nuclear parameters $\vect{a}$ such as the saturation density $n_0$, saturation energy density $\epsilon_0$, nuclear incompressibility $K_0$ etc. define a parametrized equation of state (EoS) for the neutron star.  (For this example, $\vect{p} = (n_0, \epsilon_0, K_0)$.)  Solving the Tolman–Oppenheimer–Volkoff (TOV) equation given a central pressure $P_c$ yields a set of neutron star observables including the mass $M(P_c, \vect{a})$ and radius $R(P_c, \vect{a})$ that depend on the parameters $\vect{a}$ and central pressure $P_c$.  In addition to tabulating the $M$ and $R$ as a function of $P_c$, we need to numerically compute the derivatives with respect to the parameters $\vect{a}$.

Generating this data requires repeated solution of the TOV equations over a set of (dynamically) chosen parameter values and central pressures.  These can easily be performed in parallel, hence the need for the data-format to support parallel writes.

From this information, an analysis of gravitational wave signals leads to a set of $N_p \times N_p$ Fisher information matrices $F(m_1, m_2)$ that show how observing a neutron star binary with masses $m_1$ and $m_2$ at some reference distance would constrain the parameters.  This analysis will be done in MATLAB.



## Neutron Star Properties and Parameter Dependence

In the first stage, the following information is needed:

* $\vect{a}$: Set of parameter values, associated names, and units.
* $M(P_c, \vect{a})$, $R(P_c, \vect{a})$: Table of masses and radii as a function of central pressure for this set of parameters.  Again, associated units are needed and names.
* $\partial M/\partial a_i$, $\partial R/\partial a_i$: Arrays of partial derivatives of the masses and radii as functions of the parameters.

To generate the derivatives, one needs to sample the functions $M(P_c, \vect{a})$ at a variety of points for each parameter.  For accuracy, we adaptively compute the derivatives, so for each partial, there is a list of mass evaluations at an independent set of values for that particular parameter.  A sample set of data-files might thus be:

```bash
__init__.py          # Meta-data (MD) (see below)
_classes.py          # Supporting classes used in __init__.py
params.npy           # (N_p,), double: Parameter values
stars.npy            # (3, N_m), double: P_c, M, R etc values for neutron stars
derivatives/n_0.npy  # (-, 3, N_m), double: chain derivative data for parameter n_0
            e_0.npy  # (-, 3, N_m), double: chain derivative data for parameter e_0
            K_0.npy  # (-, 3, N_m), double: chain derivative data for parameter K_0
derivatives.npy      # (3, N_m, N_p), double: Final derivatives for all parameters and all stars.
```

In this scenario, the generation code could compute the chains of derivatives for each parameter independently, enabling the code to scale to $N_p$ processes.  The final `derivatives.npy` array would have to be generated at the end once each of these processes in run.

The meta-data would be stored in the `__init__.py` file, and might look something like this:

```python
params = Array(
    filename='params.npy',
    shape=(N_p,),
    dtype=float,
    unit=(1,), 
    dimensions=[
        Dimension(
            name='param',
            coords=['n_0', 'e_0', 'K_0'],
            unit=None,
            units=['1/fm**3', 'MeV/fm**3', 'MeV'])],
)
               
stars = Array(
    filename='stars.npy', 
    shape=(3, N_m),
    dtype=float,
    unit=(1,0), 
    dimensions=[
        Dimension(
            name='observable',
            coords=['M', 'R', 'k_2'],
            unit=None,
            units=['M0', 'km', '1']),
        Dimension(
            name='Pc',
            coords=[0.1, 0.2, 0.3, ...],
            unit='MeV/fm**3',
            units=None)],
)
```

If one wanted to be able to run an independent process for each star, then one might instead have more files:

```bash
__init__.py              # Meta-data (MD) (see below)
params.npy               # (N_p,), double: Parameter values
stars.npy                # (3, N_m), double: P_c, M, R etc values for neutron stars
derivatives/n_0[0].npy   # (-, 3), double: chain derivative data for parameter n_0 for the first star
            n_0[1].npy   # (-, 3, N_m), double: chain derivative data for parameter n_0 for the second star
            ...
            n_0[N_m].npy # (-, 3, N_m), double: chain derivative data for parameter n_0 for the final star
            e_0[0].npy   # (-, 3, N_m), double: chain derivative data for parameter e_0 for the first star
            e_0[1].npy   # (-, 3, N_m), double: chain derivative data for parameter e_0 for the second star
            ...
derivatives.npy          # (3, N_m, N_p), double: Final derivatives for all parameters and all stars.
```

### Table of Fisher Information Matrices

One example arrising from analysis of LIGO gravitational wave data is a set of [Fisher Information Matrices](https://en.wikipedia.org/wiki/Fisher_information) describing how a binary neuton star merger would constrain some nuclear parameters.  This array might have a shape `(N_m, N_m, N_p, N_p)` where all pairs of neutron star masses are considered from a set of `N_m` masses, and there are `N_p` nuclear parameters.  For this particular example, we will consider four parameters: $a$, and $b$ which are in units of MeV and $\alpha$ and $\beta$ which are dimensionless.  The first two indices could be described by the following `Dimension` object:

```python
Dimension(name='mass', label='M_1', coords=[(np.linspace(0.9, 2.1, 100), Unit('M_sun')),]
Dimension(name='parameter', label='p', coords=[('a', 'b', 'alpha', 'beta'), 
                                               np.linspace(0.9, 2.1, 100), Unit('M_sun')),]          
```

## [`xarray`]

As we will often make use of the [`xarray`] package, we provide the following mapping which allows us to use our meta-data to populate an [`xarray`]:




[`xarray`](http://xarray.pydata.org)

# Existing Solutions

## [HDF5]

[HDF5] is a rather complete solution, but has several issues.

1. The binary format is somewhat complex, generally requiring external tools.  While these are generally available, it is a rather complex library, and we would like our data to be more accessible.
2. One of our motivations (see below) is to provide parallel write access to the data, and possibly to use memory mapped files.  The last time I looked into this, [HDF5] did not seem to transparently support this. (See the [Parallel HDF5 Overview](https://support.hdfgroup.org/HDF5/Tutor/poverview.html) for details: this may have changed.)
3. We do not need the full set of flexibility (and additional complexity) provided by [HDF5].

Here are some related discussions:

* [Should you use HDF5? (Cyrille Rossant)](http://cyrille.rossant.net/should-you-use-hdf5/)
* [On HDF5 and the future of data management (Konrad Hinsen)](http://blog.khinsen.net/posts/2016/01/07/on-hdf5-and-the-future-of-data-management/)

[HDF5]: https://support.hdfgroup.org/HDF5/

## [netCDF]

Built on top of [HDF5], [netCDF] adds some nice structure, but suffers from all of the limitations of using [HDF5].  The biggest motivation for using [netCDF] is that the python [`xarray`] module uses [netCDF] under the hood, and the [`holoviews`](http://holoviews.org/) plotting library uses [`xarray`], providing a nice option for viewing.

[netCDF]: https://www.unidata.ucar.edu/software/netcdf/
[HDF5]: https://support.hdfgroup.org/HDF5/
[`xarray`](http://xarray.pydata.org)

## Exdir

It appears that what we are trying to do is implemented by the [Exdir] standard, described [in this paper](https://www.biorxiv.org/content/early/2018/01/18/249979).  With an appropriate convention for metadata, datasets, etc. this might work well.  However, [Exdir] seems to require python 3 which is not an option yet, and maps pretty directly to [HDF5], which is a bit complex for our needs.

[Exdir]: https://github.com/CINPLA/exdir/
[HDF5]: https://support.hdfgroup.org/HDF5/